In [1]:
import pandas as pd
import numpy as np
from time import time

from Package.MatrixDistance import *

# Content

In this notebook, we provide a working example of using our distance, with a particular focus on what the input data should look like.

## Load the dataset

We load a dataset from the ones contained in the `Data` folder. This dataset describes proximity interactions between children in a school context. Its format is composed of five columns
* `t`: timestamp of the interaction between $i$ and $j$
* `i`: id of the first node
* `j`: id of the second node
* `C1`: school class `i` belongs to
* `C2`: school class `j` belongs to

These last two columns are unnecessary when computing the distance and will be discarded.

> **Remarks**: 
>    * In our case, the column `t` is expressed as the number of seconds from an initial time. If it was in a date-time format, it should be converted to the total number of seconds format.
>    * in the datasets we considered in the paper, all interactions are undirected, but the distance framework can be applied also to the directed setting as detailed below.

In [5]:
# load the dataset and consider only the relevant columns
df = pd.read_csv('Data/SPData/highschool_2011.csv', sep = '\t', header = None, names = ['t', 'i', 'j', 'C1', 'C2'])
df = df[['i', 'j', 't']]
df.head()

,i,j,t
0,26,66,54120
1,26,66,54140
2,24,25,54160
3,100,37,54160
4,37,57,54180


## Pre-process the time column

Here, we need to make two operations needed to maintain a high efficiency in the implementation. 

* Set the smallest entry of the `t` column to $0$
* Let all entries of the `t` column be multiples of the time resolution


In this dataset, the temporal resolution is $20~s$, so $t_{\rm res}$ can be any value larger or equal to $20$.

In [7]:
# shift the smallest time to zero
df.t = df.t - df.t.min()

# set a temporal resolution
tres = 10*60
df.t = (df.t/tres).astype(int)
df.head()

,i,j,t,τ
0,1,2,33,1
1,1,2,33,1
2,1,2,33,1
3,1,2,33,1
4,1,2,33,1


## Add the weight column

Now that we have set a temporal resolution, we group all interactions happening at the same time.

> **Remark**. Letting the weight be the number of appearances of an edge within a timestamp is an arbitrary choice. The distance definition takes a weighted temporal graph as input and attributing the weights is part of a data pre-processing step that, in general, depends on the graph under consideration. In all cases, the column `τ` should be present. When considering an unweighted graph, the column `τ` should be filled with $1$'s.

In [8]:
# add the weight column to encode the fraction 
df['τ'] = 1
df = df.groupby(['i', 'j', 't']).sum().reset_index()
df.head()

,i,j,t,τ
0,1,2,33,5
1,1,3,1,1
2,1,3,145,2
3,1,3,323,2
4,1,3,436,2


## Create a mapping for the nodes

If the input graph does not label the nodes from $0$ to $n-1$, then this mapping should be performed by hand.

> **Remark**: if you are using the *matched* distance, pay attention to using the same mapping on all graphs.

In [9]:
# all nodes present in the graph
nodes = np.unique(df[['i', 'j']])

# number of nodes
n = len(nodes)

# dictionary mapping the nodes' ids to integers
NodeMapper = dict(zip(nodes, np.arange(n)))

# map the nodes identities
df.i = df.i.map(lambda x: NodeMapper[x])
df.j = df.j.map(lambda x: NodeMapper[x])

## Example of use

Since we need two graphs to compute a distance, we will split `df` into $2$ parts for illustration purposes: one with all interactions happening before the median time and one with all interactions happening after. The argument optional `symmetric` allows you to consider directed graphs as well (see the documentation of the function).

In [10]:
# median time
tmed = df.t.median()

# all interactions happening before or at the median time
df1 = df[df.t <= tmed]

# all interactions happening after (bringing the smallest value df2.t to 0 once again)
df2 = df[df.t > tmed]
df2.t = df2.t - df2.t.min()

In [ ]:
# get the embedding
X1 = GraphDynamicEmbedding(df1, n, verbose = True)
X2 = GraphDynamicEmbedding(df2, n, verbose = True)

# compute the distance
dist = EmbDistance(X1, X2, distance_type = 'matched')
dist

Running the optimization for k = 1
Running the optimization for k = 1


43.98502101273601

In [14]:
# alternatively you can get the distance directly from one function
DynamicGraphDistance(df1, df2, n1 = n, n2 = n, distance_type = 'matched')

44.00420800674333

# Remarks

1) When adopting the embedding+distance strategy, different runs give slightly different results. This is because of the randomness in the initialization of the embeddings. In the `DynamicGraphDistance` function, we fix the seeds, and the outcome is deterministic. Consider the following example in which we compute the distance between `df1` and `df1`.

In [ ]:
# get the embedding
X1 = GraphDynamicEmbedding(df1, n, verbose = True)
X2 = GraphDynamicEmbedding(df1, n, verbose = True)
EmbDistance(X1, X2, distance_type = 'matched')

Running the optimization for k = 1
Running the optimization for k = 1


1.8140417147856016

In [16]:
# alternatively you can get the distance directly from one function
DynamicGraphDistance(df1, df1, n1 = n, n2 = n, distance_type = 'matched')

0.0

By fixing the seeds, the distance between a graph and itself is correctly equal to zero. In the first case, instead, it is not but it is much smaller than the distance that was computed in the previous example.

2. Letting the smallest entry of `df.t` be equal $0$ is only a matter of performance. Here we compute the distance between two identical graphs, in which time is shifted by a constant value. 

In [17]:
# df1 is the same as df2, but in one case the smallest time is set to 0 and in the other it is not
df2 = copy(df1)
df2.t = df2.t + 1e4
DynamicGraphDistance(df1, df2, n1 = n, n2 = n, distance_type = 'matched')

0.0